## Summary: Introducing Agentic Decisions into RAG Pipelines
Overview  
This demo expands a basic RAG (Retrieval-Augmented Generation) pipeline to include agentic decision-making. The system dynamically decides whether to rely on retrieved documents or perform live web research depending on the quality of retrieved information.

Key Steps Covered  
## 1. Setting up the Vector Store  
Documents (from a previously used PDF) are loaded into a Chroma vector database.

Embeddings are generated and stored.

This is the standard offline preprocessing step for RAG.


vectorstore = Chroma(...)

vectorstore.add_documents(...)

In [ ]:
import os 
from typing import List, Dict
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import MessagesState
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display
from tavily import TavilyClient

In [ ]:
from dotenv import load_dotenv
load_dotenv()

**Loading Documents to VectorDB**

## 2. Defining the State Schema  
A custom state is defined including:  

search_required: a yes/no flag determining whether a web search is necessary.

Other standard RAG fields (messages, question, etc.).


class State(MessageState):  

  search_required: Optional[str]

In [ ]:
embeddings_fn = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

In [ ]:
vector_store = Chroma(
    collection_name="udacity",
    embedding_function=embeddings_fn
)

file_path = "compact-guide-to-large-language-models.pdf"
loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(pages)

_ = vector_store.add_documents(documents=all_splits)

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

In [ ]:
class State(MessagesState):
    question: str
    documents: List[Document]
    search_required: str = "NO"
    answer: str

In [ ]:
def retrieve(state: State):
    question = state["question"]
    retrieved_docs = vector_store.similarity_search(question)
    return {"documents": retrieved_docs}

In [ ]:
def evaluator(state: State):
    question = state["question"]
    documents = state["documents"]
    docs_content = "\n\n".join(doc.page_content for doc in documents)
    
    template = ChatPromptTemplate([
        ("system", "You are an assistant for evaluating context."),
        ("human", "Use the following pieces of retrieved context to determine if an " 
                  "additional search is required. If the context is relevant to the " 
                  "question, your response should be 'NO', i.e. no need for search. "
                  "If the context is not relevant, say 'YES', i.e. search the web for an answer. " 
                  "Don't explain anything, if it's to search the web, say YES, otherwise, NO. "
                  "\n# Question: \n-> {question} "
                  "\n# Context: \n-> {context} "
                  "\n# Answer: "),
    ])

    chain = template | llm | StrOutputParser()

    search_required = chain.invoke(
        {"context": docs_content, "question": question}
    )

    return {"search_required": search_required}
    

In [ ]:
@tool
def web_search(question:str)->Dict:
    """
    Return top search results for a given search query
    """
    tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    response = tavily_client.search(question)
    return response

In [ ]:
llm_with_tools = llm.bind_tools([web_search])

In [ ]:
def researcher(state: State):
    question = state["question"]
    messages = state["messages"]
    if not messages:
        human_message = HumanMessage(
            "Conduct a web research to findout the answer to the following question: "
            f"```{question}```"
        )
        messages = [human_message]
    
    ai_message = llm_with_tools.invoke(messages)
    messages.append(ai_message)
    return {"messages": messages, "answer": ai_message.content}

In [ ]:
def augment(state: State):
    question = state["question"]
    documents = state["documents"]
    docs_content = "\n\n".join(doc.page_content for doc in documents)
    
    template = ChatPromptTemplate([
        ("system", "You are an assistant for question-answering tasks."),
        ("human", "Use the following pieces of retrieved context to answer the question. "
                "If you don't know the answer, just say that you don't know. " 
                "Use three sentences maximum and keep the answer concise. "
                "\n# Question: \n-> {question} "
                "\n# Context: \n-> {context} "
                "\n# Answer: "),
    ])

    messages = template.invoke(
        {"context": docs_content, "question": question}
    ).to_messages()

    return {"messages": messages}

In [ ]:
def generate(state: State):
    ai_message = llm.invoke(state["messages"])
    return {"answer": ai_message.content, "messages": ai_message}

In [ ]:
def rag_router(state: MessagesState):
    search_required = state["search_required"]
    if search_required.lower() == "yes":
        return "researcher"

    return "augment"

In [ ]:
def tool_router(state: MessagesState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "web_search"

    return END

## 3. Creating Nodes  
a. Retrieve Node  
Performs similarity search on the vector store to find relevant documents.

retrieved_docs = retriever.invoke(state.question)  
b. Evaluator Node  
Uses an LLM chain with a prompt asking:

"Based on retrieved documents, is a web search required? Answer 'yes' or 'no'."
The LLM's response sets the search_required field.


prompt = ChatPromptTemplate.from_messages([...])

evaluator = prompt | llm | StrOutputParser()  
c. Researcher Node  
If web search is needed, the researcher node uses Tavily’s web search tool.

Searches the web and updates messages with search results.


web_search_tool = tavily_client.search  
d. Augment Node  
If web search is not required, retrieved documents are passed to the LLM for direct answering.

augment_prompt = ChatPromptTemplate.from_messages([...])

augment = augment_prompt | llm | StrOutputParser()  
e. Generate Node  
Final output generation based on the selected source of context.

In [ ]:
workflow = StateGraph(State)

workflow.add_node("retrieve", retrieve)
workflow.add_node("augment", augment)
workflow.add_node("generate", generate)
workflow.add_node("evaluator", evaluator)
workflow.add_node("researcher", researcher)
workflow.add_node("web_search", ToolNode([web_search]))

workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "evaluator")

workflow.add_conditional_edges(
    source="evaluator", 
    path=rag_router, 
    path_map=["augment", "researcher"]
)

workflow.add_conditional_edges(
    source="researcher", 
    path=tool_router, 
    path_map=["web_search", END]
)
workflow.add_edge("web_search", "researcher")

workflow.add_edge("augment", "generate")
workflow.add_edge("generate", END)


In [ ]:
graph = workflow.compile()
display(
    Image(
        graph.get_graph().draw_mermaid_png()
    )
)

In [ ]:
output = graph.invoke(
    input={"question": "What is Pokemon?"},
)

In [ ]:
output["answer"]

In [ ]:
for message in output["messages"]:
    message.pretty_print()

In [ ]:
output["search_required"]

In [ ]:
output = graph.invoke(
    input={"question": "What is Open Source model?"},
)

In [ ]:
output["search_required"]

In [ ]:
for message in output["messages"]:
    message.pretty_print()

## 4. Defining Router Functions  
Two routers manage control flow:  

First router:

If search_required == "yes", route to the researcher.

If no, route to augment.


def should_search_router(state):

  return "researcher" if state.search_required == "yes" else "augment"
Second router:

After tool use, if more tool calls are needed, loop.

Otherwise, terminate.


def tool_router(state):

  ...

## 5. Building the Workflow  
Nodes: retrieve, evaluator, researcher, augment, web_search_tool

Edges:

start → retrieve → evaluator

Conditional edge based on search decision.

If web search is needed, the researcher loops until termination.

Otherwise, augment proceeds directly to answer generation.


workflow.add_conditional_edges("evaluator", should_search_router)
Workflow is visualized and compiled.

## 6. Execution Examples  
a. Question: "What is Pokémon?"  
Retrieval fails (no relevant documents).

Evaluator responds "yes" to search_required.

Researcher performs web search.

Returns correct answer: "Pokemon, short for Pocket Monsters..."

b. Question: "What is open source model?"  
Retrieval succeeds.

Evaluator responds "no" to search_required.

Context is used directly from the document without web search.

Answer successfully pulled from offline documents.

7. Key Concepts Highlighted
Dynamic decision-making inside a RAG workflow.

Fallback to web search when knowledge gaps are detected.

Agentic behavior: LLM assesses context sufficiency and adapts.

Efficient resource use: Web search is only triggered when necessary.

8. Conclusion
Adding agentic decisions to RAG pipelines increases reliability and adaptability.

Blending offline retrieval with online search creates more robust AI systems.

This modular, decision-driven design is foundational for advanced AI agents.